In [1]:
import pandas as pd
import numpy as np
from time import time
import matplotlib.pyplot as plt

from sklearn.metrics import make_scorer,accuracy_score, precision_score, recall_score,f1_score,confusion_matrix,balanced_accuracy_score
from scipy.sparse import csr_matrix,issparse
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer as Normalizer
from sklearn.preprocessing import StandardScaler, QuantileTransformer,PowerTransformer,MinMaxScaler,MaxAbsScaler,RobustScaler
from sklearn.decomposition import TruncatedSVD
from datetime import datetime 
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
%matplotlib inline
plt.style.use('fivethirtyeight')


#from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.svm import LinearSVC
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.preprocessing import Normalizer as Normalizer

In [2]:

df01=pd.read_csv(r'C:\Users\tekin\Desktop\ML\Tweedy\TechCntVctUsnm_D1906.csv',index_col=False)
#df01.drop(columns='editor',inplace=True)

In [3]:
df01.head(10)

,edInput,editor,usScName,text
0,1,5007,DigitalTrends,By_digitaltrends model crew use driveable tile...
1,1,5007,DigitalTrends,By_digitaltrends device feed Oreos without use...
2,1,5007,DigitalTrends,By_digitaltrends old version @windows critical...
3,1,5007,DigitalTrends,By_digitaltrends UK intelligent agency want tr...
4,1,5007,DigitalTrends,By_digitaltrends Cleaning pet get easy
5,1,5007,DigitalTrends,By_digitaltrends Curiosity rover find evidence...
6,1,5007,DigitalTrends,By_digitaltrends room build lego
7,1,5007,DigitalTrends,By_digitaltrends robot play pet go
8,1,5007,Ronald_vanLoon,By_ronald_vanloon #userexperience #ux heart #d...
9,1,5007,Ronald_vanLoon,By_ronald_vanloon Describe understand bayesian...


In [4]:
SEED=7
np.random.seed(SEED)


In [5]:
y_vect=np.array(df01.edInput)
text_vect=np.array(df01.text[:].values)

In [6]:
X_train_raw, X_test_raw, y_train_all, y_test =train_test_split(text_vect,
            y_vect,random_state=SEED,test_size=0.1,stratify=y_vect)

In [7]:
y_test.shape

(1439,)

In [10]:
#Use count vectorizer then tfid
#!!!!!!!!! we can add a column with a flag value that shows the tweet has pic or video into word vector

lemma_vect =  CountVectorizer(tokenizer=None,token_pattern='\S+',
                             min_df=1,lowercase=False,stop_words=None,ngram_range=(1,5),max_features=14000)
X_train=lemma_vect.fit_transform(X_train_raw)
X_test=lemma_vect.transform(X_test_raw)
print(X_train.shape,X_test.shape)

(12951, 14000) (1439, 14000)


svd= TruncatedSVD(n_components=5000)


X_train_rd = svd.fit_transform(X_train_tmp)
X_test_rd = svd.transform(X_test_tmp)
print(svd.explained_variance_ratio_.sum())

feature_names=lemma_vect.get_feature_names()
best_features = [feature_names[i] for i in svd.components_[0].argsort()[::-1]]


In [9]:
print(type(X_train))
print(type(X_test))

NameError: name 'X_train' is not defined

In [22]:
#X_train1, X_val, y_train, y_val = train_test_split(X_train, y_train_all, test_size=0.1,stratify=y_train_all)

In [8]:


del text_vect
del df01
del y_vect

In [9]:
def specificity(y_true, y_pred):
    spcfc= confusion_matrix(y_true, y_pred)[0,0]/(confusion_matrix(y_true,y_pred)[0,0] + confusion_matrix(y_true, y_pred)[0,1])
    return spcfc
def tn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 0]
def fp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 1]
def fn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 0]
def tp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 1]

FeatureRD=TruncatedSVD(n_components=1500)
quantile=QuantileTransformer(output_distribution='uniform')
powertr= PowerTransformer(method='yeo-johnson')
myscoring = {'accuracy' : make_scorer(accuracy_score),
             'balanced_acc': make_scorer(balanced_accuracy_score),
             'specificity' : make_scorer(specificity),  
           'precision' : make_scorer( precision_score),
           'f1_score' : make_scorer(f1_score),
            'tp': make_scorer(tp), 'tn': make_scorer(tn),
            'fp': make_scorer(fp), 'fn': make_scorer(fn)}

           

In [10]:

class TweetVect(CountVectorizer):
    def __init__(self,max_features=700,ngram_range=(1,3),tokenizer=None,token_pattern='\S+',min_df=1,lowercase=False,stop_words=None):
        super().__init__(tokenizer=None,token_pattern='\S+',\
                    min_df=1,lowercase=False,stop_words=None,ngram_range=ngram_range,max_features=max_features)
    def fit(self, X, y=None):

        self.fit_transform(X)
        return self

    def fit_transform(self,X,y=None):
         return super().fit_transform(X).toarray()

    def transform(self,X,y=None):
        return super().transform(X).toarray()



In [16]:

cv2 = StratifiedShuffleSplit(n_splits=5, test_size=0.1,train_size=0.9, random_state=SEED) 




pipeonl = Pipeline([('vect',TweetVect()),('prep', MinMaxScaler()),
                     ('classifier',LinearSVC(class_weight = 'balanced', max_iter=3000))
                   ])

param_one = [{  'vect__ngram_range': [(1, 5)]
                 ,'vect__max_features': [150,300,450,700,1500,2500,3500,5000]
                 ,'prep': [MinMaxScaler(),StandardScaler(with_mean=False),powertr,MaxAbsScaler()] 
                 ,'classifier': [LinearSVC(class_weight = 'balanced', max_iter=3000)] 
                 ,'classifier__C': [0.001,0.01, 0.1, 1, 10, 100]
                  }]



gridone = GridSearchCV(pipeonl, param_one, cv=cv2,scoring=myscoring,refit='balanced_acc',verbose=2,n_jobs=2)
gridone.fit(X_train_raw, y_train_all)
best_mdlone = gridone.best_estimator_

print("Best params with lemma:\n{}\n".format(gridone.best_params_))
print("Best cross-validation score with lemma: {:.5f}".format(gridone.best_score_))
print("Test-set score with lemma: {:.5f}".format(gridone.score(X_test_raw, y_test)))
y_pred = best_mdlone.predict(X_test_raw)
print("Best specificty: {:.5f}".format(specificity(y_test,y_pred)))
print("Best balanced accuracy: {:.5f}".format(balanced_accuracy_score(y_test,y_pred)))
print("Best precision: {:.5f}".format(precision_score(y_test,y_pred)))
print("Best f1_score: {:.5f}".format(f1_score(y_test,y_pred)))
print(confusion_matrix(y_test, y_pred))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print(tn,fp,fn,tp)


Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:  1.9min
[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed: 46.2min
[Parallel(n_jobs=2)]: Done 361 tasks      | elapsed: 187.9min
[Parallel(n_jobs=2)]: Done 600 out of 600 | elapsed: 382.6min finished


Best params with lemma:
{'classifier': LinearSVC(C=0.01, class_weight='balanced', dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=3000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0), 'classifier__C': 0.01, 'prep': MinMaxScaler(copy=True, feature_range=(0, 1)), 'vect__max_features': 7000, 'vect__ngram_range': (1, 5)}

Best cross-validation score with lemma: 0.78602
Test-set score with lemma: 0.76361
Best specificty: 0.72956
Best balanced accuracy: 0.76361
Best precision: 0.95959
Best f1_score: 0.87116
[[ 116   43]
 [ 259 1021]]
116 43 259 1021


In [17]:
df=pd.DataFrame(gridone.cv_results_)
df.to_csv(r'C:\Users\tekin\Desktop\ML\Tweedy\Results\gridspcfcRnPC_400to700_5000to9000_D1906.csv')

In [ ]:
#SVM with probability


In [ ]:
#create a pipeline for production use joblib or pickle to dump that pipelin without Model; check this method works

from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

X_train,X_test,y_train,y_test = make_my_dataset()

pipeline = Pipeline([
    ('vect',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('clf',LinearSVC())
])

# Instead of 'just' fitting the pipeline on the training
# data, do cross-validation too so that you know if it's
# overfitting.
# This returns an array of values, each having the score 
# for an individual run.
# - cv=3 means that we're doing 3-fold cross validation
# - You can select any metric to score your pipeline
scores = cross_val_scores(pipeline,X_train,y_train,cv=3,
    scoring='f1_micro')

#https://amueller.github.io/COMS4995-s18/slides/aml-13-022818-resampling-imbalanced-data/#9
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
scores = cross_validate(LogisticRegression(),
                        X_train, y_train, cv=10, scoring=('roc_auc', 'average_precision'))
scores['test_roc_auc'].mean(), scores['test_average_precision'].mean()

# with the information above, you can be more 
# comfortable to train on the whole dataset
pipeline.fit(X_train,y_train)

y_preds = pipeline.predict(x_test)

mean_f1 = f1_score(y_test, y_preds, average='micro')



In [ ]:
#if input vector X is sparse then use Standardscaler(mean=False)
pipe = Pipeline([('svd', FeatureRD),('prep', StandardScaler()),('classifier', LinearSVC(class_weight = 'balanced'))])

param_grid = [

                {'svd__n_components':[400,750,1500,2500,5000]
                 ,'classifier': [LinearSVC(class_weight = 'balanced')] 
                 ,'classifier__C': [0.01, 0.1, 1, 10, 100]
                 ,'prep': [StandardScaler(),MinMaxScaler(),powertr]
                 
                 
                }

             ]
cv2 = StratifiedShuffleSplit(n_splits=5, test_size=0.1,train_size=0.9, random_state=SEED) 
gridlnr = GridSearchCV(pipe, param_grid, cv=cv2,scoring=myscoring,refit='accuracy',verbose=2,n_jobs=2)
gridlnr.fit(X_train, y_train_all)
best_model = gridlnr.best_estimator_
print("Best params with lemma:\n{}\n".format(gridlnr.best_params_))
print("Best cross-validation score with lemma: {:.2f}".format(gridlnr.best_score_))


In [ ]:
from sklearn.svm import SVC
piperbf = Pipeline([('svd', FeatureRD),('prep', StandardScaler()),('classifier', SVC(class_weight = 'balanced'))])

param_gridrbf = [

                {'classifier': [SVC(class_weight = 'balanced')] 
                 ,'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]
                 ,'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100]
                 ,'prep': [StandardScaler(),MinMaxScaler(),powertr,None]
                 ,'svd__n_components':[200,400,750,1500,2500,5000]
                 
                }

             ]

gridrbf = GridSearchCV(piperbf, param_gridrbf, cv=cv2,scoring=myscoring,refit='accuracy',verbose=1,n_jobs=2)
gridrbf.fit(X_train, y_train_all)
best_model = gridrbf.best_estimator_
print("Best params with lemma:\n{}\n".format(gridrbf.best_params_))
print("Best cross-validation score with lemma: {:.2f}".format(gridrbf.best_score_))


In [ ]:
X_test_fnl=gridrbf.transform(X_test)
print("Test-set score with lemma: {:.2f}".format(gridrbf.score(X_test, y_test)))
y_pred = best_model.predict(X_test)
print("Best specificity: {:.5f}".format(specificity(y_test,y_pred)))
print("Best accuracy: {:.5f}".format(accuracy_score(y_test,y_pred)))
print("Best precision: {:.5f}".format(precision_score(y_test,y_pred)))
print("Best f1_score: {:.5f}".format(f1_score(y_test,y_pred)))


In [18]:
lemma_vect =  CountVectorizer(tokenizer=None,token_pattern='\S+',
                             min_df=1,lowercase=False,stop_words=None,ngram_range=(1,4),max_features=14000)
X_train=lemma_vect.fit_transform(X_train_raw)
X_test=lemma_vect.transform(X_test_raw)

pipetwo = Pipeline([('svd', FeatureRD),('prep', MinMaxScaler()),
                     ('classifier',LinearSVC(class_weight = 'balanced'))
                   ])

param_two = [{ 'classifier': [LinearSVC(class_weight = 'balanced')] 
                 ,'classifier__C': [10]
                 ,'svd__n_components':[1500]
                 ,'prep': [MinMaxScaler(),StandardScaler()] }]



gridtwo = GridSearchCV(pipetwo, param_two, cv=cv2,scoring=myscoring,refit='balanced_acc',verbose=2,n_jobs=2)
gridtwo.fit(X_train, y_train_all)
best_mdltwo = gridtwo.best_estimator_

print("Best params with lemma:\n{}\n".format(gridtwo.best_params_))
print("Best cross-validation score with lemma: {:.5f}".format(gridtwo.best_score_))
print("Test-set score with lemma: {:.5f}".format(gridtwo.score(X_test, y_test)))
y_pred = best_mdltwo.predict(X_test)
print("Best test specificty: {:.5f}".format(specificity(y_test,y_pred)))
print("Best test accuracy: {:.5f}".format(accuracy_score(y_test,y_pred)))
print("Best test balanced accuracy: {:.5f}".format(balanced_accuracy_score(y_test,y_pred)))
print("Best test precision: {:.5f}".format(precision_score(y_test,y_pred)))
print("Best test f1_score: {:.5f}".format(f1_score(y_test,y_pred)))
print(confusion_matrix(y_test, y_pred))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print(tn,fp,fn,tp)



Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:  7.3min finished
c:\users\tekin\appdata\local\programs\python\python36\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Best params with lemma:
{'classifier': LinearSVC(C=10, class_weight='balanced', dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0), 'classifier__C': 10, 'prep': MinMaxScaler(copy=True, feature_range=(0, 1)), 'svd__n_components': 1500}

Best cross-validation score with lemma: 0.85833
Test-set score with lemma: 0.85962
Best test specificty: 0.43396
Best test accuracy: 0.85962
Best test balanced accuracy: 0.67323
Best test precision: 0.92846
Best test f1_score: 0.92041
